# Outil d'Annotation Manuelle des Résumés



In [ ]:
import pandas as pd
import json
import random
from pathlib import Path
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Chemins
BASE_DIR = Path().resolve().parent.parent
RESULTS_DIR = BASE_DIR / "data" / "results"
ANNOTATIONS_DIR = RESULTS_DIR / "human_annotations"
ANNOTATIONS_DIR.mkdir(exist_ok=True)

In [ ]:
# Charger les résumés générés
with open(RESULTS_DIR / "all_summaries_and_scores.json", "r", encoding="utf-8") as f:
    data = json.load(f)

summaries = data["summaries"]
evaluations = data["evaluations"]

print(f"Nombre total de résumés : {len(summaries)}")

In [ ]:
class AnnotationInterface:
    def __init__(self, summaries_data, evaluations_data, sample_size=25):
        self.summaries = summaries_data
        self.evaluations = evaluations_data
        self.sample_size = sample_size
        self.current_index = 0
        self.annotations = []
        
        # Sélection stratifiée : mix scores élevés/moyens/faibles
        self.selected_indices = self._stratified_sample()
        
        # Widgets
        self.setup_widgets()
        
    def _stratified_sample(self):
        """Échantillonnage stratifié basé sur les scores composites"""
        scores = [(i, eval_data.get('Score composite', 0.5)) 
                 for i, eval_data in enumerate(self.evaluations)]
        scores.sort(key=lambda x: x[1])
        
        # Prendre 1/3 de chaque catégorie
        n_per_group = self.sample_size // 3
        low_scores = scores[:len(scores)//3][:n_per_group]
        mid_scores = scores[len(scores)//3:2*len(scores)//3][:n_per_group]
        high_scores = scores[2*len(scores)//3:][:n_per_group]
        
        selected = [x[0] for x in low_scores + mid_scores + high_scores]
        random.shuffle(selected)
        return selected
    
    def setup_widgets(self):
        # Critères d'évaluation
        self.factuality_slider = widgets.IntSlider(
            value=3, min=1, max=5, description='Factualité:', 
            tooltip='1=Très incorrecte, 5=Parfaitement correcte'
        )
        
        self.coherence_slider = widgets.IntSlider(
            value=3, min=1, max=5, description='Cohérence:',
            tooltip='1=Incohérente, 5=Très cohérente'
        )
        
        self.readability_slider = widgets.IntSlider(
            value=3, min=1, max=5, description='Lisibilité:',
            tooltip='1=Très difficile, 5=Très facile à lire'
        )
        
        self.completeness_slider = widgets.IntSlider(
            value=3, min=1, max=5, description='Complétude:',
            tooltip='1=Manque beaucoup, 5=Très complet'
        )
        
        self.overall_slider = widgets.IntSlider(
            value=3, min=1, max=5, description='Global:',
            tooltip='1=Très mauvais, 5=Excellent résumé'
        )
        
        # Zone de commentaires
        self.comments_text = widgets.Textarea(
            placeholder='Commentaires optionnels (erreurs spécifiques, suggestions...)'
        )
        
        # Boutons
        self.prev_button = widgets.Button(description='Précédent', disabled=True)
        self.next_button = widgets.Button(description='Suivant ')
        self.save_button = widgets.Button(description=' Sauvegarder', button_style='success')
        
        # Events
        self.prev_button.on_click(self.prev_summary)
        self.next_button.on_click(self.next_summary)
        self.save_button.on_click(self.save_annotation)
        
        # Progress
        self.progress = widgets.IntProgress(
            value=0, min=0, max=len(self.selected_indices),
            description='Progrès:'
        )
        
    def display_current_summary(self):
        if self.current_index >= len(self.selected_indices):
            display(HTML("<h3> Annotation terminée ! Merci !</h3>"))
            return
            
        idx = self.selected_indices[self.current_index]
        summary_data = self.summaries[idx]
        eval_data = self.evaluations[idx]
        
        # Affichage du résumé et texte source
        html_content = f"""
        <div style='border: 2px solid #e1e5e9; padding: 15px; margin: 10px 0; border-radius: 8px;'>
            <h4> Article #{self.current_index + 1}/{len(self.selected_indices)} (ID: {summary_data['summary_id']})</h4>
            
            <div style='background: #f8f9fa; padding: 10px; margin: 10px 0; border-radius: 5px;'>
                <strong> Résumé généré :</strong><br>
                {summary_data['ensemble_summary']['summary']}
            </div>
            
            <div style='background: #e8f4fd; padding: 10px; margin: 10px 0; border-radius: 5px; max-height: 300px; overflow-y: auto;'>
                <strong> Article original (extrait) :</strong><br>
                {summary_data.get('source_text', 'Non disponible')[:1000]}...
            </div>
            
            <div style='background: #fff3cd; padding: 8px; margin: 10px 0; border-radius: 5px;'>
                <strong> Scores automatiques :</strong> 
                Factualité: {eval_data.get('Factualité', 'N/A'):.3f} | 
                Cohérence: {eval_data.get('Cohérence', 'N/A'):.3f} | 
                Score global: {eval_data.get('Score composite', 'N/A'):.3f}
            </div>
        </div>
        """
        
        clear_output(wait=True)
        display(HTML(html_content))
        
        # Widgets d'annotation
        display(widgets.VBox([
            self.progress,
            widgets.HTML("<h4> Votre évaluation :</h4>"),
            self.factuality_slider,
            self.coherence_slider, 
            self.readability_slider,
            self.completeness_slider,
            self.overall_slider,
            self.comments_text,
            widgets.HBox([self.prev_button, self.next_button, self.save_button])
        ]))
        
        # Mise à jour des boutons
        self.prev_button.disabled = (self.current_index == 0)
        self.next_button.disabled = (self.current_index >= len(self.selected_indices) - 1)
        self.progress.value = self.current_index
        
    def save_annotation(self, button):
        idx = self.selected_indices[self.current_index]
        
        annotation = {
            'summary_id': self.summaries[idx]['summary_id'],
            'original_index': idx,
            'timestamp': datetime.now().isoformat(),
            'human_scores': {
                'factuality': self.factuality_slider.value,
                'coherence': self.coherence_slider.value,
                'readability': self.readability_slider.value,
                'completeness': self.completeness_slider.value,
                'overall': self.overall_slider.value
            },
            'comments': self.comments_text.value,
            'automatic_scores': {
                'factuality': self.evaluations[idx].get('Factualité'),
                'coherence': self.evaluations[idx].get('Cohérence'),
                'composite': self.evaluations[idx].get('Score composite')
            }
        }
        
        self.annotations.append(annotation)
        
        # Sauvegarde immédiate
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = ANNOTATIONS_DIR / f"annotations_{timestamp}.json"
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.annotations, f, ensure_ascii=False, indent=2)
            
        print(f" Annotation {self.current_index + 1} sauvegardée !")
        
    def next_summary(self, button):
        if self.current_index < len(self.selected_indices) - 1:
            self.current_index += 1
            self.display_current_summary()
            
    def prev_summary(self, button):
        if self.current_index > 0:
            self.current_index -= 1
            self.display_current_summary()

# Initialiser l'interface
annotation_tool = AnnotationInterface(summaries, evaluations, sample_size=25)
annotation_tool.display_current_summary()

In [ ]:
# Analyse des annotations collectées
def analyze_annotations():
    annotation_files = list(ANNOTATIONS_DIR.glob("annotations_*.json"))
    if not annotation_files:
        print("Aucune annotation trouvée.")
        return
        
    # Charger la dernière annotation
    latest_file = max(annotation_files, key=lambda x: x.stat().st_mtime)
    with open(latest_file, 'r', encoding='utf-8') as f:
        annotations = json.load(f)
    
    if not annotations:
        print("Aucune annotation dans le fichier.")
        return
        
    # Créer DataFrame pour analyse
    df_annotations = pd.DataFrame([
        {
            'summary_id': ann['summary_id'],
            'human_factuality': ann['human_scores']['factuality'],
            'human_coherence': ann['human_scores']['coherence'],
            'human_readability': ann['human_scores']['readability'],
            'human_overall': ann['human_scores']['overall'],
            'auto_factuality': ann['automatic_scores']['factuality'],
            'auto_coherence': ann['automatic_scores']['coherence'],
            'auto_composite': ann['automatic_scores']['composite'],
            'comments': ann['comments']
        }
        for ann in annotations
    ])
    
    print(f" Analyse de {len(df_annotations)} annotations")
    print("\n=== MOYENNES SCORES HUMAINS ===")
    human_cols = ['human_factuality', 'human_coherence', 'human_readability', 'human_overall']
    print(df_annotations[human_cols].mean())
    
    print("\n=== CORRÉLATIONS HUMAIN vs AUTOMATIQUE ===")
    correlations = {
        'Factualité': df_annotations[['human_factuality', 'auto_factuality']].corr().iloc[0,1],
        'Cohérence': df_annotations[['human_coherence', 'auto_coherence']].corr().iloc[0,1],
        'Global': df_annotations[['human_overall', 'auto_composite']].corr().iloc[0,1]
    }
    
    for metric, corr in correlations.items():
        print(f"{metric}: {corr:.3f}")
        
    return df_annotations

# Lancer l'analyse si des annotations existent
df_human = analyze_annotations()